### 아이디어
- 쿼리에서 LCA를 찾고 각 정점에서 LCA까지의 최대, 최소 도로 길이를 찾는다.
- `DP[i][j] = i번 정점에서 2^j번째 조상까지의 최대, 최소 도로 길이`
- A에서 LCA까지의 최대/최소값 vs B에서 LCA까지의 최대/최소값

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

N = int(input())
NONE = -1
T = [[] for _ in range(N+1)]
for _ in range(N-1) :
  u, v, w = map(int, input().split())
  T[u].append((v, w))
  T[v].append((u, w))

def dfs(T, P, LO, HI, L) :
  vis = [False] * (N+1)
  vis[1] = True
  Q = [1]
  l = 1
  while Q :
    nextQ = []
    while Q:
      u = Q.pop()
      for v, w in T[u] :
        if w != NONE and not vis[v] :
          vis[v] = True
          P[0][v] = u
          LO[0][v] = w
          HI[0][v] = w
          L[v] = l
          nextQ.append(v)
    Q = nextQ
    l += 1

MAX = 17
L = [0] * (N+1)
P = [[0] * (N+1) for _ in range(MAX)] #DP[i][j] : j의 2^i번째 부모
LO = [[int(2e9)] * (N+1) for _ in range(MAX)] #[i][j] : j의 2^i번째 부모까지 거리의 최대/최소 거리
HI = [[0] * (N+1) for _ in range(MAX)]
dfs(T, P, LO, HI, L)

for i, l in enumerate(P[1:], 1) :
  for j in range(N+1) :
    tmp = P[i-1][j]
    l[j] = P[i-1][tmp]
    LO[i][j] = min(LO[i-1][j], LO[i-1][tmp]) # 현재 - 부모 거리 vs 부모 - 부모의 부모 거리
    HI[i][j] = max(HI[i-1][j], HI[i-1][tmp])

for _ in range(int(input())) :
  a, b = map(int, input().split())
  
  if L[a] < L[b] :
    a, b = b, a
  
  a_lo = b_lo = int(2e9)
  a_hi = b_hi = 0
  for i in range(MAX-1, -1, -1) : #높이의 차이를 logN만에 올림
    if L[a] - L[b] >= (1 << i) :
      a_lo = min(a_lo, LO[i][a])
      a_hi = max(a_hi, HI[i][a])
      a = P[i][a] #커서를 옮기기 전에 최대/최소 거리를 갱신해둬야 한다.

  if a != b:
    for i in range(MAX-1, -1, -1) :
      if P[i][a] != -1 and P[i][a] != P[i][b] :
        a_lo = min(a_lo, LO[i][a])
        a_hi = max(a_hi, HI[i][a])
        a = P[i][a]
        b_lo = min(b_lo, LO[i][b])
        b_hi = max(b_hi, HI[i][b])
        b = P[i][b]
    a_lo = min(a_lo, LO[i][a])
    a_hi = max(a_hi, HI[i][a])
    b_lo = min(b_lo, LO[i][b])
    b_hi = max(b_hi, HI[i][b])
  
  
  low = min(a_lo, b_lo)
  high = max(a_hi, b_hi)
  sys.stdout.write(f"{low} {high}\n")

- 아이디어는 맞았다

# 피드백
- 그래프의 연결정보를 matrix로 저장하는 것은 신중을 가해야 한다. (N+1) * (N+1)로 초기화했더니 메모리 오버가 났다.
- LCA를 찾을 때 부모를 옮기기 전(`a = P[i][a]` 하기 전)에, 필요한 계산을 다 마쳐야 한다.
  - 이 문제에서 최대값과 최소값을 구하는 과정(`a_lo = min(a_lo, LO[i][a])` 등)을 부모를 옮기기 전에 해서 원하는 결과가 나오지 않았다.
- 이 문제는 LCA를 출력할 필요가 없으므로 마지막에 `P[i][a]`하는 과정을 추가로 하지 않아도 된다.
  - 다만 부모가 된 정점의 최대와 최소값을 구해야 하기 때문에, 최하단 루프문의 바깥에서 각각 최소/최대 값을 구해주었다.